# Language Modeling using Ngram

In this Exercise, you are going to use NLTK which is a natural language processing library for python to create a bigram language model and its variation. You will build one model for each of the following type and calculate their perplexity:
- Unigram Model
- Bigram Model
- Bigram Model with Laplace smoothing
- Bigram Model with Interpolation
- Bigram Model with Kneser-ney Interpolation
- Neural LM (optional)



In [ ]:
#download corpus
!wget --no-check-certificate https://github.com/ekapolc/nlp_2019/raw/master/HW4/BEST2010.zip
!unzip BEST2010.zip

--2021-02-14 14:12:10--  https://github.com/ekapolc/nlp_2019/raw/master/HW4/BEST2010.zip
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ekapolc/nlp_2019/master/HW4/BEST2010.zip [following]
--2021-02-14 14:12:11--  https://raw.githubusercontent.com/ekapolc/nlp_2019/master/HW4/BEST2010.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7423530 (7.1M) [application/zip]
Saving to: ‘BEST2010.zip’

BEST2010.zip        100%[===================>]   7.08M  27.5MB/s    in 0.3s    

2021-02-14 14:12:11 (27.5 MB/s) - ‘BEST2010.zip’ saved [7423530/7423530]

Archive:  BEST2010.zip
   creating: BEST2010/
  i

In [ ]:
#First we import necessary library such as math, nltk, bigram, and collections.
import math
import nltk
import io
import random
from random import shuffle
from nltk import bigrams, trigrams
from collections import Counter, defaultdict
random.seed(999)

BEST2010 is a free Thai NLP dataset by NECTEC usually use as a standard benchmark for various NLP tasks includeing language modeling. BEST2010 is separated into 4 domain article, encyclopedia, news and novel. The data is already  tokenized using '|' as a separator.

For example,

ตาม|ที่|นางประนอม ทองจันทร์| |กับ| |ด.ช.กิตติพงษ์ แหลมผักแว่น| |และ| |ด.ญ.กาญจนา กรองแก้ว| |ป่วย|สงสัย|ติด|เชื้อ|ไข้|ขณะ|นี้|ยัง|ไม่|ดี|ขึ้น|

In [ ]:
# We choose news domain as our dataset
best2010=[]
fp= io.open('BEST2010/news.txt','r',encoding='utf-8')
for i,line in enumerate(fp):
    best2010.append(line.strip()[:-1])
fp.close()
all_vocabulary =set()
total_word_count =0
for line in best2010:
    for word in line.split('|'):        
        all_vocabulary.add(word)
        total_word_count+=1

In [ ]:
#For simplicity, we assumes that each line is a sentence.
print ('Total sentences in BEST2010 news dataset :\t'+ str(len(best2010)))
print ('Total word counts in BEST2010 news dataset :\t'+ str(total_word_count))
print ('Total vocabulary in BEST2010 news dataset :\t'+ str(len(all_vocabulary)))

Total sentences in BEST2010 news dataset :	30969
Total word counts in BEST2010 news dataset :	1660190
Total vocabulary in BEST2010 news dataset :	35488


We separate out input into 2 sets, train and test data with 70:30 ratio

In [ ]:
sentences = best2010
# The data is separated to train and test set with 70:30 ratio.
train = sentences[:int(len(sentences)*0.7)]
test = sentences[int(len(sentences)*0.7):]

#Training data
train_vocabulary =set()
train_word_count =0
for line in train:
    for word in line.split('|'):        
        train_vocabulary.add(word)
        train_word_count+=1
print ('Total sentences in BEST2010 news training dataset :\t'+ str(len(train)))
print ('Total word counts in BEST2010 news training dataset :\t'+ str(train_word_count))
print ('Total vocabuary in BEST2010 news training dataset :\t'+ str(len(train_vocabulary)))
# We will use 1/vocab_size as a default value for unknown word
unk_value = math.pow(len(train_vocabulary),-1)

Total sentences in BEST2010 news training dataset :	21678
Total word counts in BEST2010 news training dataset :	1042797
Total vocabuary in BEST2010 news training dataset :	26240


# Unigram

In this section, we will demonstrate how to build a unigram language model <br>
**Important note:** <br>
**\<s\>** = sentence start symbol <br>
**\</s\>** = sentence end symbol 

In [ ]:
def getUnigramModel(data):
    model = defaultdict(lambda: 0)
    word_count =0
    for sentence in data:
        sentence +=  u'|</s>' #for unigram model we can always ignore <s>, since p(w0=<s>)=1
        for w1 in sentence.split('|'):
            model[w1] +=1.0
            word_count+=1
    for w1 in model:
        model[w1] = model[w1]/(word_count)
    return model

In [ ]:
model = getUnigramModel(train)

In [ ]:
def getLnValue(x):
    if x >0.0:
        return math.log(x)
    else:
        return math.log(unk_value)

In [ ]:
#problability of 'นายก'
print(getLnValue(model[u'นายก']))
#for example, problability of 'นายกรัฐมนตรี' which is an unknown word is equal to
print(getLnValue(model[u'นายกรัฐมนตรี']))
#problability of 'นายก' 'ได้' 'ให้' 'สัมภาษณ์' 'กับ' 'สื่อ'
prob = getLnValue(model[u'นายก'])+getLnValue(model[u'ได้'])+ getLnValue(model[u'ให้'])+getLnValue(model[u'สัมภาษณ์'])+getLnValue(model[u'กับ'])+getLnValue(model[u'สื่อ'])+getLnValue(model['</s>'])
print ('Problability of a sentence', math.exp(prob))

-6.551526663995246
-10.175040243058024
Problability of a sentence 5.617210748667918e-18


# Perplexity

In order to compare language model we need to calculate perplexity. In this task you should write a perplexity calculation code for the unigram model. The result perplexity should be around 556.39 and
476.07 on train and test data.

# TODO #0
**Fill your name and ID here** <br>
**Name**: Phollawat Janesiripanich
<br>
**ID**: 6130348121

## TODO #1 **Calculate perplexity**

In [ ]:
import numpy as np
def calculate_sentence_ln_prob(sentence, model):
    sentence_ln_prob=0
    sentence_list = sentence.split("|")
    for word in sentence_list:
      sentence_ln_prob=sentence_ln_prob+getLnValue(model[word])
    return sentence_ln_prob

def perplexity(test,model):
    N_word = 0
    sum_ln = 0
    for sentence in test:
      sentence_with_stop = sentence+"|</s>"
      sum_ln = sum_ln + calculate_sentence_ln_prob(sentence_with_stop,model)
      N_word = N_word + len(sentence_with_stop.split("|"))
    return np.exp(-sum_ln/N_word)
      

In [ ]:
print(perplexity(train,model))
print(perplexity(test,model))
a= 556.3925994212195
b= 476.0687892303532
#if(perplexity(train,model)==a):
#  print("yes")
#if(perplexity(test,model)==b):
#  print("yes")

556.3925994212195
476.0687892303532


## TODO #2 **Please explain why this model give us such a high perplexity.**

**Your answer**:  
Because in this model we only use unigram to calculate each word probabiltys, that mean every word could possibly after each other which will make the number of time to guess the right sentence become so high and by the meaning of perplexity which is the number of time to guess each word to the correct sentecnce. So it mean that a perplexity is high.

# Bigram

Next, you will create a better language model than a unigram (which is not much to compare with). But first, it is very tedious to count every pair of words that occur in our corpus by ourselves. In this case, nltk provide us a simple library which will do it for us.

In [ ]:
#example of nltk usage for bigram
sentence = 'I always search google for an answer .'

print('This is how nltk generate bigram.')
for w1,w2 in bigrams(sentence.split(), pad_right=True, pad_left=True):
    print (w1,w2)
print('None is used as a start and end of sentence symbol.')

This is how nltk generate bigram.
None I
I always
always search
search google
google for
for an
an answer
answer .
. None
None is used as a start and end of sentence symbol.


Now, you should be able to implement a bigram model by yourself. Also, you must create a new perplexity calculation for bigram. The result perplexity should be around 58.54 and 145.88 on train and test data.

## TODO #3 **Create a Bigram Model**

In [ ]:
def getBigramModel(data):
    ###FILL YOUR CODE HERE###
    unigram_model = defaultdict(lambda: 0)
    bigram_model = defaultdict(lambda: defaultdict(lambda: 0))

    for sentence in data:
      for now,next in bigrams(sentence.split('|'), pad_right=True, pad_left=True):
          unigram_model[now]=unigram_model[now]+1.0
          bigram_model[now][next]=bigram_model[now][next]+1.0

    for now in unigram_model:
      for next in bigram_model[now]:
        bigram_model[now][next]=bigram_model[now][next]/unigram_model[now]

    return bigram_model

model = getBigramModel(train)



## TODO #4 **Calculate Perplexity for Bigram Model**



In [ ]:
def calculate_sentence_ln_prob(sentence, model):
    sentence_ln_prob=0
    for now,next in bigrams(sentence.split('|'), pad_right=True, pad_left=True):
      sentence_ln_prob+=getLnValue(model[now][next])
    return sentence_ln_prob

def perplexity(test,model):
    N_word = 0
    sum_ln = 0
    for sentence in test:
      sum_ln = sum_ln + calculate_sentence_ln_prob(sentence,model)
      N_word = N_word + len(sentence.split("|"))+1
    return np.exp(-sum_ln/N_word)
   

In [ ]:
print (perplexity(train,model) )
print (perplexity(test, model))

a=58.78942889767147
b=146.26539331038614
#if(perplexity(train,model)==a):
#  print("yes")
#if(perplexity(test,model)==b):
#  print("yes")

58.78942889767147
146.26539331038614


# Smoothing

Usually any ngram models have a sparsity problem, which means it does not have every possible ngram of words in the dataset. Smoothing techniques can alleviate this problem. In this section, you will implement two basic smoothing methods laplace smoothing and interpolation for bigram.

## TODO #5 **Bigram with Laplace smoothing (Add-One Smoothing)**

In [ ]:
#Laplace Smoothing
def getBigramWithLaplaceSmoothing(data):
    #Fill code here
    unigram_model = defaultdict(lambda: 0)
    bigram_model = defaultdict(lambda: defaultdict(lambda: 0))

    for sentence in data:
      for now,next in bigrams(sentence.split('|'), pad_right=True, pad_left=True):
          unigram_model[now]=unigram_model[now]+1.0
          bigram_model[now][next]=bigram_model[now][next]+1.0

    for now in unigram_model:
      for next in bigram_model[now]:
        bigram_model[now][next]=(bigram_model[now][next]+1)/(unigram_model[now]+len(unigram_model))

    return bigram_model

model = getBigramWithLaplaceSmoothing(train)
print (perplexity(train,model) )
print (perplexity(test, model))

a= 974.8134581679766
b= 1098.1622194979489
#if(perplexity(train,model)==a):
#  print("yes")
#if(perplexity(test,model)==b):
#  print("yes")

974.8134581679766
1098.1622194979489


## TODO #6 **Bigram with Interpolation**
lambda value is 0.7 for bigram, 0.25 for unigram, and 0.05 for unknown word

In [ ]:
#interpolation
def getBigramWithInterpolation(data):
    #Fill code here
    unigram_model = defaultdict(lambda: 0)
    bigram_model = defaultdict(lambda: defaultdict(lambda: 0))
    total_word=0

    for sentence in data:
      for now,next in bigrams(sentence.split('|'), pad_right=True, pad_left=True):
          unigram_model[now]=unigram_model[now]+1.0
          bigram_model[now][next]=bigram_model[now][next]+1.0
    for vocab in unigram_model:
      total_word=total_word+unigram_model[vocab]
    total_word=total_word-unigram_model[None]
    for now in unigram_model:
      for next in bigram_model[now]:
        bigram_model[now][next]=(0.7*bigram_model[now][next]/unigram_model[now])+(0.25*unigram_model[next]/total_word)+(0.05/len(unigram_model))

    return bigram_model
    
model = getBigramWithInterpolation(train)
print (perplexity(train,model))        
print (perplexity(test,model))
a= 73.38409869825665
b= 172.67485908813356
#if(perplexity(train,model)==a):
#  print("yes")
#if(perplexity(test,model)==b):
#  print("yes")

73.38410880445593
172.67487186736156


the result perplexity on training and testing should be 

    974.81, 1098.16 for Laplace smoothing
    73.38, 172.67 for Interpolation

# Language modeling on multiple domains

Sometimes, we do not have enough data to create a language model for a new domain. In that case, we can improvised by combining several models to improve result on the new domain.

In this exercise you will try to merge two language models from news and article domains to create a language model for the encyclopedia domain.

In [ ]:
# create article data
encyclo_data=[]
fp= io.open('BEST2010/encyclopedia.txt','r',encoding='utf-8')
for i,line in enumerate(fp):
    encyclo_data.append(line.strip()[:-1])
fp.close()

First, you should try to calculate perplexity of your bigram with interpolation using "news data" (train) on "encyclopedia data" (test). The result perplexity should be around 727.35.

For your information, a bigram model with interpolation using "ariticle data" (train) to test on "encyclopedia data" (test) has a perplexity of 505.79.

In [ ]:
#print perplexity of bigram with interpolation on article data        
print (perplexity(encyclo_data,model))


727.3503259715216


## TODO #7 
Write a model that produce 450.0 or less perplexity on encyclopedia data without using data from the encyclopedia as training data. (Hint : Try to combine a model with news data and a model with article data together.)

In [ ]:
# Fill code here
article_data=[]
fp= io.open('BEST2010/article.txt','r',encoding='utf-8')
for i,line in enumerate(fp):
    article_data.append(line.strip()[:-1])
fp.close()

train = article_data+sentences
combined_model = getBigramWithInterpolation(train)
print('Perplexity of combine Bigram model with interpolation smoothing on encyclopedia test data',perplexity(encyclo_data, combined_model))

Perplexity of combine Bigram model with interpolation smoothing on encyclopedia test data 427.50913889506825


## TODO #8 
## Kneser-ney on "News"

<!-- Reimplement equation 4.33 in SLP textbook (https://lagunita.stanford.edu/c4x/Engineering/CS-224N/asset/slp4.pdf) -->

Implement Bigram Knerser-ney LM. The result perplexity should be around 55.06 and 140.76 on train and test data. 


In [ ]:
# Fill codehere
train = sentences[:int(len(sentences)*0.7)]
test = sentences[int(len(sentences)*0.7):]
def getBigramWithKneserNey(data):
    #Fill code here
    unigram_model = defaultdict(lambda: 0)
    bigram_model = defaultdict(lambda: defaultdict(lambda: 0))
    KneserNey_model = defaultdict(lambda: defaultdict(lambda: 0))
    possible_past = defaultdict(lambda: 0)
    total_word=0
    possible_word=0
    
    for sentence in data:
      for now,next in bigrams(sentence.split('|'), pad_right=True, pad_left=True):
          unigram_model[now]=unigram_model[now]+1.0
          bigram_model[now][next]=bigram_model[now][next]+1.0

    for vocab in unigram_model:
      total_word=total_word+unigram_model[vocab]
    total_word=total_word-unigram_model[None]

    for vocab in bigram_model:
      possible_word=possible_word+len(bigram_model[vocab])

    for now in bigram_model:
      for next in bigram_model[now]:
        possible_past[next]= possible_past[next]+1

    for now in unigram_model:
      for next in bigram_model[now]:
        KneserNey_model[now][next]=(max(bigram_model[now][next]-0.75,0)/unigram_model[now])+(0.75*len(bigram_model[now])/unigram_model[now])*(possible_past[next]/ possible_word)

    return KneserNey_model

model = getBigramWithKneserNey(train)

print (perplexity(train,model))        
print (perplexity(test,model))
# 71.14054002208687
# 174.02464248000433


71.14054002208687
155.09274968738495


## TODO #9
## Neural LM (optional)
do it on news corpus that we splitted into train and test sets at the beginning of this exercise. 

In [ ]:
#there are many ways to do this. e.g.:
#https://machinelearningmastery.com/develop-word-based-neural-language-models-python-keras/
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google_drive_downloader import GoogleDriveDownloader as gdd
import tensorflow as tf
%matplotlib inline
gdd.download_file_from_google_drive(file_id='1iodAqVNWEkiJgH8cWkccsLi_tqoFcMrV',
                                    dest_path='./corpora.tar.gz')
!tar xvzf corpora.tar.gz


corpora/
corpora/mnist_data/
corpora/mnist_data/t10k-images-idx3-ubyte.gz
corpora/mnist_data/train-images-idx3-ubyte.gz
corpora/mnist_data/.ipynb_checkpoints/
corpora/mnist_data/vis_utils.py
corpora/mnist_data/__init__.py
corpora/mnist_data/load_mnist.py
corpora/mnist_data/train-labels-idx1-ubyte.gz
corpora/mnist_data/t10k-labels-idx1-ubyte.gz
corpora/BEST/
corpora/BEST/test/
corpora/BEST/test/df_best_article_test.csv
corpora/BEST/test/df_best_encyclopedia_test.csv
corpora/BEST/test/df_best_novel_test.csv
corpora/BEST/test/df_best_news_test.csv
corpora/BEST/train/
corpora/BEST/train/df_best_encyclopedia_train.csv
corpora/BEST/train/df_best_article_train.csv
corpora/BEST/train/df_best_news_train.csv
corpora/BEST/train/df_best_novel_train.csv
corpora/BEST/val/
corpora/BEST/val/df_best_encyclopedia_val.csv
corpora/BEST/val/df_best_news_val.csv
corpora/BEST/val/df_best_article_val.csv
corpora/BEST/val/df_best_novel_val.csv
corpora/.ipynb_checkpoints/
corpora/.ipynb_checkpoints/Word_Tokeniz

In [ ]:
from keras.preprocessing.text import Tokenizer
data = """ Jack and Jill went up the hill\n
		To fetch a pail of water\n
		Jack fell down and broke his crown\n
		And Jill came tumbling after\n """

# integer encode text
tokenizer = Tokenizer()
tokenizer.fit_on_texts([train[0]])

encoded = tokenizer.texts_to_sequences([train[0]])[0]
print(tokenizer,[train[0]],encoded)
for i in range(10):
  print(train[i])
